In [1]:
import pandas as pd
from itertools import chain
import regex as re
from underthesea import word_tokenize


In [2]:
df = pd.read_csv("../data/procedure.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2038 entries, 0 to 2037
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   procedure_name  2038 non-null   object
dtypes: object(1)
memory usage: 16.0+ KB


In [3]:
# new_df = df.drop_duplicates()
# new_df.to_csv('../data/new_procedure.csv', index=False)

In [4]:
# df = pd.read_csv('../data/new_procedure.csv')
# df.head()

,procedure_name
0,Thủ tục chuyển nhượng toàn bộ hoặc một phần dự...
1,Thủ tục cấp mới chứng chỉ hành nghề môi giới b...
2,Thủ tục cấp lại (cấp đổi) chứng chỉ hành nghề ...
3,Thủ tục thông báo nhà ở hình thành trong tương...
4,"Thủ tục cho thuê, thuê mua nhà ở xã hội thuộc ..."


In [15]:
# f = open('../data/ongia_VA.txt', 'r', encoding='utf-8')
# f_new = open('../data/procedure.txt', 'w', encoding='utf-8')
# procedures = f.readlines()
# for procedure in procedures:
#     procedure = procedure.replace('\n','')
#     f_new.write(procedure+'\t'+procedure)
#     f_new.write('\n')
# f_new.close()


In [9]:
df = pd.read_csv('../data/new_procedure.csv')

In [10]:
df.head()

,procedure_name
0,thủ tục chuyển nhượng toàn bộ hoặc một phần dự...
1,thủ tục cấp mới chứng chỉ hành nghề môi giới b...
2,thủ tục cấp lại (cấp đổi) chứng chỉ hành nghề ...
3,thủ tục thông báo nhà ở hình thành trong tương...
4,"thủ tục cho thuê, thuê mua nhà ở xã hội thuộc ..."


In [11]:
procedure_list = df.procedure_name.values.tolist()

In [12]:
procedure_list[:5]

['thủ tục chuyển nhượng toàn bộ hoặc một phần dự án bất động sản do ubnd cấp tỉnh quyết định việc đầu tư',
 'thủ tục cấp mới chứng chỉ hành nghề môi giới bất động sản',
 'thủ tục cấp lại (cấp đổi) chứng chỉ hành nghề môi giới bất động sản do bị mất, bị rách, bị cháy, bị hủy hoại do thiên tai hoặc lý do bất khả kháng hoặc cấp lại chứng chỉ hành nghề môi giới bất động sản do hết hạn (hoặc gần hết hạn)',
 'thủ tục thông báo nhà ở hình thành trong tương lai đủ điều kiện được bán, cho thuê mua',
 'thủ tục cho thuê, thuê mua nhà ở xã hội thuộc sở hữu nhà nước']

In [14]:
tokens = []

In [15]:

for line in procedure_list:
    tokens += word_tokenize(line)
vocab = list(set(tokens))

In [16]:
vocab[:5]

['nhập cảnh', 'ban hành', 'sxtn', 'bỏ rơi', 'gia hạn']

In [9]:
def lcs(X, Y):
    # find the length of the strings
    m = len(X)
    n = len(Y)

    # declaring the array for storing the dp values
    L = [[None] * (n + 1) for i in range(m + 1)]

    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])

    return L[m][n]


In [16]:
def load_dict_values(dict_file):
    fin = open(dict_file, 'r', encoding='utf-8')
    lines = fin.readlines()
    fin.close()
    values_dict = []
    for line in lines:
        # print('line : ', line)
        items = line.strip().split('\t')
        # print('items : ', items)
        if len(items) == 2:
            words = items[0].lower().split()
            values_dict.append((words, items[1]))
    return values_dict


In [ ]:
list_procedure = load_dict_values('../data/procedure.txt')
list_procedure[:5]


In [19]:
def find_best_matching_in_dict(query, values_dict):
    query_words = query.lower().split()
    max_ratio = 0
    str_found = ""
    for words, v in values_dict:
        r = 1.0 * lcs(words, query_words) / len(words)
        if r > max_ratio:
            max_ratio = r
            str_found = v
    return str_found


In [61]:
mingg = find_best_matching_in_dict('kết hôn lại', load_dict_values('../data/procedure.txt'))
mingg

'Thủ tục đăng ký kết hôn'

In [19]:
def search_in_database(user_token):
    df = pd.read_csv('../data/new_procedure.csv')
    procedures = df[df.procedure_name.str.contains(user_token, na=False)]
    # flatten 2d list
    procedure_list = list(chain.from_iterable(procedures.values.tolist()))
    procedure_list = sorted(procedure_list, key=len)
    return procedure_list


In [69]:
def remove_dup(result_list):
    return list(dict.fromkeys(result_list))


In [20]:
kh = search_in_database('kết hôn')
kh

['thủ tục đăng ký kết hôn',
 'thủ tục đăng ký lại kết hôn',
 'thủ tục đăng ký kết hôn lưu động',
 'thủ tục đăng ký kết hôn có yếu tố nước ngoài',
 'thủ tục đăng ký lại kết hôn có yếu tố nước ngoài',
 'thủ tục ghi vào sổ hộ tịch việc kết hôn của công dân việt nam đã được giải quyết tại cơ quan có thẩm quyền của nước ngoài',
 'thủ tục ghi vào sổ hộ tịch việc ly hôn, hủy việc kết hôn của công dân việt nam đã được giải quyết tại cơ quan có thẩm quyền của nước ngoài']

In [17]:
user_sent = "đăng ký kết hôn"
tmp = [user_sent]
tmp

['đăng ký kết hôn']

In [75]:
def ranking_result(user_sent:str):
    values_dict = load_dict_values('../data/procedure.txt')
    best_matching = find_best_matching_in_dict(user_sent, values_dict)
    user_tokens = word_tokenize(user_sent)
    tmp =[]
    for token in user_tokens:
        tmp += search_in_database(token)
    unique_list = []
    dup_list = []
    for element in tmp:
        if element not in unique_list:
            unique_list.append(element)
        else:
            dup_list.append(element)
    dup_list = remove_dup(dup_list)
    for element in dup_list:
        if element in unique_list:
            unique_list.remove(element)
    if len(dup_list) == 0:
        return unique_list[:5]
    elif len(dup_list) <5 :
        n = 5 - len(dup_list)
        if best_matching:
            result = [best_matching] + dup_list + unique_list[:10]
            result = remove_dup(result)
            return result[:5]
        else :
            result = dup_list + unique_list[:5-n]
            return result[:5]
    else:
        if best_matching in dup_list:
            result = [best_matching] + dup_list
            result = remove_dup(result)
            return result[:5]
        else:
            return dup_list[:5]

In [78]:
mingg = ranking_result('khai sinh lại đăng ký')
mingg

['Thủ tục đăng ký lại khai sinh',
 'Thủ tục Đăng ký lại khai sinh có yếu tố nước ngoài',
 'Thủ tục đăng ký khai sinh',
 'Thủ tục đăng ký lại khai tử',
 'Thủ tục đăng ký lại kết hôn']

In [23]:
len(vocab)

1742

In [33]:
vocab_file = open('../data/vocab.txt','w',encoding='utf-8')
for token in vocab:
    vocab_file.write(token)
    vocab_file.write('\n')
vocab_file.close()

In [69]:
vocab_file = open('../data/vocab.txt', 'r', encoding='utf-8')
vocab = vocab_file.readlines()


In [70]:
len(vocab)

1632

In [71]:
_dict = {}

In [72]:
for token in vocab:
    token = token.replace('\n','')
    frequency = len(search_in_database(token))
    _dict.update({token: frequency})

In [73]:
_dict

{'nhập cảnh': 1,
 'ban hành': 5,
 'sxtn': 1,
 'bỏ rơi': 1,
 'gia hạn': 56,
 'phục vụ': 18,
 'năng khiếu': 2,
 'công-ten-nơ': 2,
 'thời điểm': 2,
 'kết hợp': 3,
 'kiến trúc': 6,
 'hình thành': 4,
 'trò chơi': 10,
 'nhằm': 5,
 'viên chức': 22,
 'ưu đãi': 28,
 'mang': 5,
 'atvstp01': 1,
 'nghĩa trang': 2,
 'tốt nghiệp': 3,
 'kinh doanh': 153,
 'mảnh': 3,
 'xét nghiệm': 5,
 'mức sống': 1,
 'campuchia': 10,
 'lâp': 1,
 'việc làm': 11,
 'giao kết': 2,
 'môi trường': 26,
 'thể hình': 1,
 'sót': 6,
 'công chứng nhận': 2,
 'phụ thuộc': 1,
 'kê': 12,
 'nhưng': 12,
 'nhà tu hành': 3,
 'nguy cấp': 3,
 'tranh hoành tráng': 17,
 'duy nhất': 1,
 'nút': 4,
 'thủy lợi': 8,
 'lpg': 15,
 'sáp nhập': 33,
 'kết cấu hạ tầng': 5,
 'thủ công': 3,
 'lĩnh vực': 6,
 'này': 4,
 'sản phẩm': 26,
 'bảo trợ': 3,
 'quyết': 147,
 'ươm': 1,
 'khuyến mại': 4,
 'phiếu': 7,
 'kết thúc': 1,
 'vật thể': 2,
 'thương binh': 17,
 'điện áp': 4,
 'trang trại': 2,
 'số lượng': 4,
 'góp': 20,
 'nghiện': 9,
 'chủ lực': 1,
 'điều khi

In [74]:
keys = []
values = []
for key in _dict.keys():
    keys.append(key)
for value in _dict.values():
    values.append(value)


In [54]:
print(keys[:10])
values[:10]


['nhập cảnh', 'ban hành', 'sxtn', 'bỏ rơi', 'gia hạn', 'phục vụ', 'năng khiếu', 'công-ten-nơ', 'thời điểm', 'kết hợp']


[1, 5, 1, 1, 56, 18, 2, 2, 2, 3]

In [75]:
d = {'token': keys, 'frequency': values}
df = pd.DataFrame(data=d)
df.head()


,token,frequency
0,nhập cảnh,1
1,ban hành,5
2,sxtn,1
3,bỏ rơi,1
4,gia hạn,56


In [76]:
df.to_csv('../data/token_frequency.csv',index=False)

In [77]:
df = pd.read_csv('../data/token_frequency.csv')
df.head()

,token,frequency
0,nhập cảnh,1
1,ban hành,5
2,sxtn,1
3,bỏ rơi,1
4,gia hạn,56


In [78]:
df = df.sort_values(by='frequency', ascending=False)

In [79]:
df.head(10)

,token,frequency
1079,thủ,966
126,thủ tục,896
1328,cấp,818
88,. cấp,579
921,giấy,578
1102,ăn,457
902,nhận,446
716,ở,436
1013,của,426
463,cô,423


In [80]:
df.to_csv('../data/token_frequency.csv',index=False)